In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
conn = sqlite3.connect('../db.sqlite3')
table1 = pd.read_sql('SELECT * FROM restaurant_order', conn)
table2 = pd.read_sql('SELECT * FROM restaurant_orderitem', conn)
table3 = pd.read_sql('SELECT * FROM restaurant_product', conn)
combined_df = pd.concat([table1, table2, table3], ignore_index=True)
combined_df.to_csv('combined_data.csv', index=False)
final_df = pd.read_csv('combined_data.csv')
conn.close()

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
df = pd.read_csv('combined_data.csv')
product_sales = df['product'].value_counts().head(10)
plt.figure(figsize=(8, 8))
plt.pie(product_sales, labels=product_sales.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
plt.title('Top 10 Most Popular Products by Quantity')
plt.show()

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df = pd.read_csv('combined_data.csv')
df['Item Price'] = df['product_price'] * df['quantity']
product_revenue = df.groupby('product')['Item Price'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(8, 8))
plt.pie(product_revenue, labels=product_revenue.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
plt.title('Top 10 Products by Revenue (Item Price)')
plt.show()

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df = pd.read_csv('combined_data.csv')
df['Order Datetime'] = pd.to_datetime(df['order_datetime'])
df['Order Hour'] = df['Order Datetime'].dt.hour
df['Order Price'] = df['product_price'] * df['quantity']

income_by_hour = df.groupby('Order Hour')['Order Price'].sum()

plt.figure(figsize=(10, 6))
income_by_hour.plot(kind='bar', color='lightblue', edgecolor='black')
plt.title('Total Restaurant Income by Hour of Order')
plt.xlabel('Hour of Day')
plt.ylabel('Total Income')
plt.xticks(range(24), rotation=0)
plt.tight_layout()
plt.show()

# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
df = pd.read_csv('combined_data.csv')
df['Order Datetime'] = pd.to_datetime(df['order_datetime'])
df['Order Day Of The Week'] = df['Order Datetime'].dt.day_name()
df['Order Price'] = df['product_price'] * df['quantity']

income_by_day = df.groupby('Order Day Of The Week')['Order Price'].sum()

order_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
income_by_day = income_by_day[order_days]

plt.figure(figsize=(10, 6))
income_by_day.plot(kind='bar', color='lightgreen', edgecolor='black')
plt.title('Total Restaurant Income by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Income')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()